<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Projects/blob/main/Alzheimer_prj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
pip install tensorflow opencv-python matplotlib

<div style="direction:rtl">
<font color='green' size="5px">
 از لینک زیر دیتاست بیماری آلزایمر را دانلود کرده و در پوشه هم مسیر همین ژوپیتر نوت بوک قرار دهید
    </font>
</div>

## https://www.kaggle.com/datasets/uraninjo/augmented-alzheimer-mri-dataset

# 2. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
#data = tf.keras.utils.image_dataset_from_directory('data\OriginalDataset')

<div style="direction:rtl">
<font color='green' size="5px">
tf.keras.utils.image_dataset_from_directory
    :این خط فرمان دارای متد زیر میباشد که در صورت نیاز میتوانید هنگام فراخوانی دیتا از آنها استفاده کنید
    </font>
    <font color='red' size="4px">
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256)
    </font>
</div>

<div style="direction:rtl">
<font color='green' size="5px">
بطور مثال دیتا را مانند زیر فراخوانی میکنیم
    </font>
</div>

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data\OriginalDataset', image_size=(45, 45))

<div style="direction:rtl">
<font color='green' size="5px">
آماده سازی دیتا
    </font>
</div>

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
#images array
batch[0].shape

In [ ]:
#labels array
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 3. Scale Data

<div style="direction:rtl">
<font color='green' size="5px">
 نرمالسازی دیتا
    </font>
</div>

In [ ]:
data = data.map(lambda x,y: (x/255, tf.keras.utils.to_categorical(y, 4)))

<div style="direction:rtl">
<font color='red' size="4px">
توسط خط کد بالا، همزمان مقادیر پیکسل تصاویر با تقسیم بر 255 به بازه بین 0 تا 1 میرود و همچنین با استفاده از تابع
    tf.keras.utils.to_categorical
    با روش
    one hot
    مقادیر لیبل تصاویر را نیز نرمالسازی میکنیم تا آماده استفاده در شبکه شود.
    </font>
</div>

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# 4. Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
print(len(train))
print(len(val))
print(len(test))

# 5. Build Deep Learning Model

<div style="direction:rtl">
<font color='green' size="5px">
در این مرحله یک شبکه کانولوشنال طراحی میشود و نتایج آن بررسی میشود. در مرحله بعد با استفاده از مفاهیم یادگیری انتقالی از وزنهای یک مدل از پیش آموزش دیده استفاده خواهیم کرد
    </font>
</div>

In [ ]:
model_cnn = Sequential()

model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(45, 45, 3)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())

model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(4, activation='softmax'))

model_cnn.compile(optimizer=tf.optimizers.Adadelta(), loss=tf.losses.categorical_crossentropy, metrics=['accuracy'])

model_cnn.summary()

# 6. Train Model

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist=model_cnn.fit(train, epochs=100, validation_data=val, callbacks=[tensorboard_callback])

# 7. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 8. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model_cnn.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 9. Test

In [ ]:
import cv2

img = cv2.imread('data/OriginalDataset/MildDemented/26 (23).jpg')
plt.imshow(img)
plt.show()

In [ ]:
def pred_alza(img):
  resize = tf.image.resize(img, (45,45))
  yhat=new_model.predict(np.expand_dims(resize/255, 0))
  id_label = []
  for i in yhat[0]:
    if i < yhat[0].max():
      id_label.append(0)
    else:
      id_label.append(1)

  id_label = id_label
  name_label = ['MildDemented','ModerateDemented','NonDemented', 'VeryMildDemented']
  temp = list(zip(id_label, name_label))
  for i in range(len(temp)):
    if temp[i][0]==1:
      label = temp[i][1]

  return(label)

<div style="direction:rtl">
<font color='green' size="5px">
همانطور که مشخص است، با این ساختار، بالاترین دقتی که بدست می آید کمتر از 50 درصد میباشد که غیر قابل قبول است.
    </font>
</div>

# 10. Transfer learning

<div style="direction:rtl">
<font color='green' size="5px">
همانطور که از نتیجه تست مدل مشخص شد، استفاده از چند لایه شبکه کانولوشنال جواب مطلوب نمیدهد. به همین خاطر طبق مطالب آموزشی که در سایت بصورت رایگان موجود است، در این قسمت از مفهوم یادگیری انتقالی و مدل
    DenseNet201
    استفاده خواهیم کرد.
    </font>
</div>

In [ ]:
import tensorflow as tf
from keras.applications import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(45, 45, 3), pooling=None)

In [ ]:
x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)  # Add your custom dense layers here

predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the model
model.compile('adam', loss=tf.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
logdir='logs_resnet'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist_DenseNet=model.fit(train, epochs=100, validation_data=val, callbacks=[tensorboard_callback])

# 11. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist_DenseNet.history['loss'], color='teal', label='loss')
plt.plot(hist_DenseNet.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist_DenseNet.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist_DenseNet.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 12. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model_resnet.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

<div style="direction:rtl">
<font color='green' size="5px">
همانطور که مشخص است، با یادگیری انتقالی، بالاترین دقتی که بدست می آید حداقل 85 درصد میباشد که قابل قبول است.
    </font>
</div>

# 13. Test

In [ ]:
import cv2

img = cv2.imread('data/OriginalDataset/MildDemented/26 (23).jpg')
plt.imshow(img)
plt.show()

In [ ]:
def pred_alza(img):
  resize = tf.image.resize(img, (45,45))
  yhat=new_model.predict(np.expand_dims(resize/255, 0))
  id_label = []
  for i in yhat[0]:
    if i < yhat[0].max():
      id_label.append(0)
    else:
      id_label.append(1)

  id_label = id_label
  name_label = ['MildDemented','ModerateDemented','NonDemented', 'VeryMildDemented']
  temp = list(zip(id_label, name_label))
  for i in range(len(temp)):
    if temp[i][0]==1:
      label = temp[i][1]

  return(label)

# 14. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('/','image_alz_classifier.h5'))

# 15. Load Model

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np

In [ ]:
new_model = load_model('image_alz_classifier.h5')

<div style="direction:rtl">
<font color='green' size="5px">
فایل مدل آموزش داده شده که با پسوند
    h5
    در داخل فایل موجود است. آن را لود کرده و مثالها را بررسی نمایید
    </font>
</div>

In [ ]:
def pred_alza(img):
  resize = tf.image.resize(img, (45,45))
  yhat=new_model.predict(np.expand_dims(resize/255, 0))
  id_label = []
  for i in yhat[0]:
    if i < yhat[0].max():
      id_label.append(0)
    else:
      id_label.append(1)

  id_label = id_label
  name_label = ['MildDemented','ModerateDemented','NonDemented', 'VeryMildDemented']
  temp = list(zip(id_label, name_label))
  for i in range(len(temp)):
    if temp[i][0]==1:
      label = temp[i][1]

  return(label)

In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('data_for_test_models/OriginalDataset/MildDemented/26 (19).jpg')
label = pred_alza(img)
plt.imshow(img)
plt.title(label)
plt.show()

In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('data_for_test_models/OriginalDataset/ModerateDemented/27 (2).jpg')
label = pred_alza(img)
plt.imshow(img)
plt.title(label)
plt.show()

In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('data_for_test_models/OriginalDataset/NonDemented/26 (62).jpg')
label = pred_alza(img)
plt.imshow(img)
plt.title(label)
plt.show()

In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('data_for_test_models/OriginalDataset/VeryMildDemented/26 (44).jpg')
label = pred_alza(img)
plt.imshow(img)
plt.title(label)
plt.show()

<div style="direction:rtl">
<font color='green' size="5px">
همانطور که از مثالها مشخص است، مدل بسیار دقیق تشخیص را انجام میدهد
    </font>
</div>